In [1681]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../src')
from functions import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, chi2_contingency
from scipy.stats import ttest_ind

# Loading the Data:

In [1682]:
df_clients_profile = pd.read_csv('../Data/Raw/df_final_demo.txt')
df_web_data_1 = pd.read_csv('../Data/Raw/df_final_web_data_pt_1.txt')
df_web_data_2 = pd.read_csv('../Data/Raw/df_final_web_data_pt_2.txt')
df_experiment_clients = pd.read_csv('../Data/Raw/df_final_experiment_clients.txt')
pd.set_option('display.max_columns', None) # To display all columns
df_web_data = pd.concat([df_web_data_1, df_web_data_2], ignore_index= True)

# Exploratory Data Analysis (EDA):

## Initial Exploration

In [1683]:
df_clients_profile.head(10)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
5,3727881,5.0,71.0,30.5,U,2.0,23915.60,0.0,3.0
6,272934,5.0,66.0,58.5,U,2.0,27021.42,2.0,5.0
7,388801,30.0,361.0,57.5,M,5.0,522498.72,1.0,4.0
8,285619,30.0,369.0,67.5,M,2.0,299388.72,3.0,6.0
9,8198645,15.0,189.0,54.5,F,2.0,382303.83,6.0,9.0


In [1684]:
df_clients_profile.shape

(70609, 9)

In [1685]:
df_experiment_clients.head(10)

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
5,9466803,Test
6,3170143,Test
7,6651403,Control
8,5887227,Test
9,2105948,Control


In [1686]:
df_experiment_clients.shape

(70609, 2)

In [1687]:
df_web_data.head(10)

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


In [1688]:
df_web_data.shape

(755405, 5)

### Exploring numerical and categorical variables

In [1689]:
# Retrieving the unique data types present in the dataframe columns
df_clients_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


In [1690]:
df_experiment_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  70609 non-null  int64 
 1   Variation  50500 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [1691]:
df_web_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755405 entries, 0 to 755404
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     755405 non-null  int64 
 1   visitor_id    755405 non-null  object
 2   visit_id      755405 non-null  object
 3   process_step  755405 non-null  object
 4   date_time     755405 non-null  object
dtypes: int64(1), object(4)
memory usage: 28.8+ MB


In [1692]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_profile = df_clients_profile.select_dtypes(include=[np.number]).columns
print(numerical_columns_profile)

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')


In [1693]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_web = df_web_data.select_dtypes(include=[np.number]).columns
print(numerical_columns_web)

Index(['client_id'], dtype='object')


In [1694]:
# Extracting column names with numerical data types from the dataframe
numerical_columns_exp_cli = df_web_data.select_dtypes(include=[np.number]).columns
print(numerical_columns_exp_cli)

Index(['client_id'], dtype='object')


In [1695]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_profile = df_clients_profile.select_dtypes("number").loc[:, df_clients_profile.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_profile

""
0
1
2
3
4
...
70604
70605
70606
70607


In [1696]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_web= df_web_data.select_dtypes("number").loc[:, df_web_data.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_web

""
0
1
2
3
4
...
755400
755401
755402
755403


In [1697]:
# Separating between discrete and continuous variables, as discrete ones could potentially be treated as categorical.
# Remember to adjust the threshold (in this case, < 20) based on your dataset's specific characteristics and domain knowledge.
potential_categorical_from_numerical_exp_cli= df_experiment_clients.select_dtypes("number").loc[:, df_experiment_clients.select_dtypes("number").nunique() < 5]
potential_categorical_from_numerical_exp_cli

""
0
1
2
3
4
...
70604
70605
70606
70607


In [1698]:
# Retrieving column names with object (typically string) data types from the dataframe
df_clients_profile.select_dtypes("object").columns

Index(['gendr'], dtype='object')

In [1699]:
# Retrieving column names with object (typically string) data types from the dataframe
df_web_data.select_dtypes("object").columns

Index(['visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

In [1700]:
# Retrieving column names with object (typically string) data types from the dataframe
df_experiment_clients.select_dtypes("object").columns

Index(['Variation'], dtype='object')

In [1701]:
df_clients_profile.columns

Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [1702]:
df_clients_profile['gendr'] = df_clients_profile['gendr'].replace(['U', 'X'], 'Other')

## Drop duplicates values

In [1703]:
#clean df_clients_profile
df_clients_profile = print_clean_data(df_clients_profile)

Before
client_id            0
clnt_tenure_yr      14
clnt_tenure_mnth    14
clnt_age            15
gendr               14
num_accts           14
bal                 14
calls_6_mnth        14
logons_6_mnth       14
dtype: int64
------------------------------------------
------------------------------------------
After
client_id           0
clnt_tenure_yr      0
clnt_tenure_mnth    0
clnt_age            0
gendr               0
num_accts           0
bal                 0
calls_6_mnth        0
logons_6_mnth       0
dtype: int64


In [1704]:
#clean df_experiment_clients
df_experiment_clients = print_clean_data(df_experiment_clients)

Before
client_id        0
Variation    20109
dtype: int64
------------------------------------------
To many NaN values in column: Variation
------------------------------------------
After
client_id        0
Variation    20109
dtype: int64


In [1705]:
#clean df_web_data
df_web_data = print_clean_data(df_web_data)

Before
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64
------------------------------------------


------------------------------------------
After
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64


In [1706]:
df_web_data.head(20)

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


# Data Typing/Formatting

In [1707]:
df_clients_profile.head(10)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,Other,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,Other,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,Other,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
5,3727881,5.0,71.0,30.5,Other,2.0,23915.60,0.0,3.0
6,272934,5.0,66.0,58.5,Other,2.0,27021.42,2.0,5.0
7,388801,30.0,361.0,57.5,M,5.0,522498.72,1.0,4.0
8,285619,30.0,369.0,67.5,M,2.0,299388.72,3.0,6.0
9,8198645,15.0,189.0,54.5,F,2.0,382303.83,6.0,9.0


In [1708]:
df_clients_profile.dtypes

client_id             int64
clnt_tenure_yr      float64
clnt_tenure_mnth    float64
clnt_age            float64
gendr                object
num_accts           float64
bal                 float64
calls_6_mnth        float64
logons_6_mnth       float64
dtype: object

In [1709]:
df_experiment_clients.dtypes

client_id     int64
Variation    object
dtype: object

In [1710]:
df_web_data.dtypes

client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object

In [1711]:
df_web_data['date_time'] = pd.to_datetime(df_web_data['date_time'], format='%Y-%m-%d %H:%M:%S')

In [1712]:
df_web_data.dtypes

client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object

In [1713]:
# Convert specified columns to integer data type
df_clients_profile['clnt_tenure_yr'] = df_clients_profile['clnt_tenure_yr'].astype(int)
df_clients_profile['clnt_tenure_mnth'] = df_clients_profile['clnt_tenure_mnth'].astype(int)
df_clients_profile['clnt_age'] = df_clients_profile['clnt_age'].astype(int)
df_clients_profile['num_accts'] = df_clients_profile['num_accts'].astype(int)
df_clients_profile['calls_6_mnth'] = df_clients_profile['calls_6_mnth'].astype(int)
df_clients_profile['logons_6_mnth'] = df_clients_profile['logons_6_mnth'].astype(int)

In [1714]:
df_clients_profile.dtypes

client_id             int64
clnt_tenure_yr        int64
clnt_tenure_mnth      int64
clnt_age              int64
gendr                object
num_accts             int64
bal                 float64
calls_6_mnth          int64
logons_6_mnth         int64
dtype: object

## Create a data frame with the clients that won't participe in the experiment.

In [1715]:
#setting aside the clients that wont be part if the experiment
null_df_experiment_clients = df_experiment_clients[df_experiment_clients['Variation'].isnull()]
null_df_experiment_clients

,client_id,Variation
50500,5459747,NaN
50501,8031000,NaN
50502,1847030,NaN
50503,9713157,NaN
50504,7775828,NaN
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


## Merge the clients in the web data

### Identify the clients part of the test group

In [1716]:
#id of clients that are part of the test
df_experiment_clients_Test = df_experiment_clients[df_experiment_clients['Variation'] == 'Test']
df_experiment_clients_Test.head(10)

,client_id,Variation
0,9988021,Test
1,8320017,Test
3,1982004,Test
5,9466803,Test
6,3170143,Test
8,5887227,Test
10,438567,Test
11,2519971,Test
13,5407849,Test
16,5308474,Test


### df_test: combine visit_id with visitor_id and reoganize the columns.

In [1717]:
final_df_Test = pd.merge(df_experiment_clients_Test, df_clients_profile, on='client_id', how='inner')
final_df_Test = pd.merge(df_web_data, final_df_Test, on='client_id', how='inner')
df_test = final_df_Test
df_test.head(15)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5,64,79,Other,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5,64,79,Other,2,189023.86,1,4
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5,64,79,Other,2,189023.86,1,4
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5,64,79,Other,2,189023.86,1,4
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5,64,79,Other,2,189023.86,1,4
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,Test,5,64,79,Other,2,189023.86,1,4
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01,Test,5,64,79,Other,2,189023.86,1,4
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22,Test,5,64,79,Other,2,189023.86,1,4
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05,Test,22,274,34,M,2,36001.90,5,8
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43,Test,22,274,34,M,2,36001.90,5,8


In [1718]:
df_test[['process_step', 'Variation']].value_counts()

process_step  Variation
start         Test         56452
step_1        Test         38753
step_2        Test         30978
step_3        Test         25888
confirm       Test         25716
Name: count, dtype: int64

### Create the csv file to df_test

In [1719]:
df_test.to_csv('../Data/Cleaned_Data/df_test.csv', index=False)

### df_control: combine visit_id with visitor_id and reoganize the columns.

In [1720]:
#id of clients that are part of the control group
df_experiment_clients_Control = df_experiment_clients[df_experiment_clients['Variation'] == 'Control']
df_experiment_clients_Control.head(10)

,client_id,Variation
2,4033851,Control
4,9294070,Control
7,6651403,Control
9,2105948,Control
12,9814849,Control
14,7286739,Control
15,4446011,Control
20,4229407,Control
24,2842481,Control
27,1218661,Control


### Create the csv file to df_control

In [1722]:
df_control.to_csv('../Data/Cleaned_Data/df_control.csv', index=False)

# Pivot and create new columns

### df_test

In [1723]:
df_test.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5,64,79,Other,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5,64,79,Other,2,189023.86,1,4


In [1724]:
# Assuming df_control is your DataFrame
# 1. Data Preparation:
df_test['date_time'] = pd.to_datetime(df_test['date_time'])
df_test['date'] = df_test['date_time'].dt.date
# Combine visit_id and visitor_id into a new column
df_test['visit_visitor_id'] = df_test['visit_id'].astype(str) + "_" + df_test['visitor_id'].astype(str)

In [1725]:
# 2. Calculate time differences per step and make them positive:
# We need to sort the dataframe before calculating the time difference
df_test = df_test.sort_values(by=['visit_id', 'client_id', 'date_time'])

df_test['next_date_time'] = df_test.groupby(['visit_id', 'client_id'])['date_time'].shift(-1)

# Calculate time spent in seconds, but make sure it's always positive
df_test['time_diff_seconds'] = (df_test['next_date_time'] - df_test['date_time']).dt.total_seconds()
df_test['time_diff_minutes'] = df_test['time_diff_seconds'] / 60



# Get Last Step:
# Calculate last_step *before* aggregating data 
last_step_df = df_test.groupby('visit_visitor_id')['process_step'].last().reset_index()
last_step_df = last_step_df.rename(columns={'process_step': 'last_step'})


In [1726]:

# 3. Calculate the number of steps per visit:
df_test = pd.concat([df_test, pd.get_dummies(df_test['process_step'], prefix='count')], axis=1)
df_steps_sum = df_test.groupby(by=['client_id', 'visit_id', 'visitor_id'])[['count_confirm', 
'count_start', 'count_step_1', 'count_step_2', 'count_step_3']].agg('sum')

In [1727]:
# 4. Pivot the table to have one row per visit:
df_pivot = df_test.pivot_table(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='time_diff_minutes', fill_value=0, aggfunc='sum')
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None  # Remove the 'process_step' header
test_time_counts = pd.merge(df_pivot, df_steps_sum, on=['client_id', 'visitor_id', 'visit_id'])


In [1729]:
# 5. Merge the data with the clients' profile and get initial and final dates:
final_df_Test = pd.merge(test_time_counts, df_experiment_clients_Test, on='client_id', how='inner')
final_df_Test = pd.merge(final_df_Test, df_clients_profile, on='client_id', how='inner')

df_test_date = df_web_data.groupby(by = ['client_id', 'visit_id', 'visitor_id']).agg({'date_time': ['min', 'max']})
df_test_date.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in df_test_date.columns]
final_df_Test = pd.merge(df_test_date, final_df_Test, on=['client_id', 'visit_id', 'visitor_id'], how='inner')

# Ensure visit_visitor_id exists in final_df_Test for the merge:
final_df_Test['visit_visitor_id'] = final_df_Test['visit_id'].astype(str) + "_" + final_df_Test['visitor_id'].astype(str)  
# Calculate total time spent in the website:
# Calculate time spent in seconds, but make sure it's always positive
df_test['time_diff_seconds'] = (df_test['next_date_time'] - df_test['date_time']).dt.total_seconds()
df_test['time_diff_minutes'] = df_test['time_diff_seconds'] / 60
# Merge with last_step_df:
final_df_Test = pd.merge(final_df_Test, last_step_df, on='visit_visitor_id', how='left')

# Extract the date from date_timemax into a new column called date
final_df_Test['date'] = final_df_Test['date_timemax'].dt.date

# Remove unnecessary columns 
final_df_Test = final_df_Test.drop(columns=['visit_id', 'visitor_id'])  # Remove only visit_id and visitor_id


# Rename columns:
final_df_Test = final_df_Test.rename(columns={
    'date_timemin': 'initial_date', 
    'date_timemax': 'final_date', 
    'start': 'start_time', 
    'confirm': 'time_completion', 
    'count_confirm': 'completion',
    'count_start': 'start_step', 
    'count_step_1': '1st_step', 
    'count_step_2': '2nd_step', 
    'count_step_3': '3rd_step',
    'Variation': 'variation'
})
# Total navigations between start and last step (including start and last):
final_df_Test['navigations_bt_start_last'] = final_df_Test.apply(
    lambda row: row['start_step'] + row['1st_step'] + row['2nd_step'] + row['3rd_step'] + 1, 
    axis=1  # Add 1 for the last_step itself
)


In [1676]:
# 6. Create the final order of columns:
old_order = ['client_id', 'visit_visitor_id', 'time_completion',
       'start_time', 'step_1', 'step_2', 'step_3', 'completion', 'start_step',
       '1st_step', '2nd_step', '3rd_step', 'variation', 'clnt_tenure_yr',
       'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal',
       'calls_6_mnth', 'logons_6_mnth', 'visit_visitor_id', 'last_step',
       'date', 'navigations_bt_start_last']
new_order = ['client_id', 'visit_visitor_id', 'start_time', 'step_1', 'step_2', 'step_3','time_completion',
       'navigations_bt_start_last', 'completion', 
       'start_step','1st_step', '2nd_step', '3rd_step',  'last_step', 'variation', 
       'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal',
       'calls_6_mnth', 'logons_6_mnth',
       'date', 'initial_date', 'final_date']
df_test_final = final_df_Test[new_order]

In [ ]:
df_test_final.to_csv('../Data/Cleaned_Data/df_test_final.csv', index=False)

### Create the csv file to df_test_final

In [126]:
df_test_final.to_csv('../Data/Cleaned_Data/df_test_final.csv', index=False)

# Combine df_test and df_control

In [127]:
df_combined = pd.concat([df_test, df_control]).reset_index(drop=True)
df_combined.to_csv('../Data/Cleaned_Data/df_combined.csv', index=False)

### df_control

In [128]:
numerical_cols = ['navigations_bt_start_last']
correlations = df_test_final[numerical_cols + ['completion']].corr()['completion'].drop('completion')
correlations

navigations_bt_start_last    0.237913
Name: completion, dtype: float64

In [129]:
df_control.head(10)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:03,Control,12,149,63,M,2,142642.26,5,8
1,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:01,Control,12,149,63,M,2,142642.26,5,8
2,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:28:52,Control,12,149,63,M,2,142642.26,5,8
3,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:26:08,Control,12,149,63,M,2,142642.26,5,8
4,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:24:43,Control,12,149,63,M,2,142642.26,5,8
5,4033851,762728880_76361333336,949661017_22392791362_127391,step_1,2017-04-05 12:24:12,Control,12,149,63,M,2,142642.26,5,8
6,4033851,762728880_76361333336,949661017_22392791362_127391,start,2017-04-05 12:24:04,Control,12,149,63,M,2,142642.26,5,8
7,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:26,Control,12,149,63,M,2,142642.26,5,8
8,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:10:26,Control,12,149,63,M,2,142642.26,5,8
9,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:10:25,Control,12,149,63,M,2,142642.26,5,8


In [130]:
# Assuming df_control is your DataFrame
# 1. Data Preparation:
df_control['date_time'] = pd.to_datetime(df_control['date_time'])
df_control['date'] = df_control['date_time'].dt.date
# Combine visit_id and visitor_id into a new column
df_control['visit_visitor_id'] = df_control['visit_id'].astype(str) + "_" + df_control['visitor_id'].astype(str)

In [131]:
# 2. Calculate time differences per step and make them positive:
# We need to sort the dataframe before calculating the time difference
df_control = df_control.sort_values(by=['visit_id', 'client_id', 'date_time'])

df_control['next_date_time'] = df_control.groupby(['visit_id', 'client_id'])['date_time'].shift(-1)

# Calculate time spent in seconds, but make sure it's always positive
df_control['time_diff_seconds'] = (df_control['next_date_time'] - df_control['date_time']).dt.total_seconds()
df_control['time_diff_minutes'] = df_control['time_diff_seconds'] / 60

In [132]:
# 3. Pivot to Create Step Columns:
df_pivot = df_control.pivot_table(index=['client_id', 'visitor_id', 'visit_id'], columns='process_step', values='time_diff_minutes', fill_value=0, aggfunc='sum')
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None  # Remove the 'process_step' header

In [133]:
# 4. Rename Columns & Calculate Total Time:
df_pivot = df_pivot.rename(columns={'confirm': 'confirm_time', 'start': 'start_time', 'step_1': 'step_1', 'step_2': 'step_2', 'step_3': 'step_3'})

df_pivot['total_time_visit'] = df_pivot[['start_time', 'step_1', 'step_2', 'step_3', 'confirm_time']].sum(axis=1)

# 5. Merge with Original Data and Select/Reorder Columns:
final_df_c = pd.merge(df_pivot, df_control[['client_id', 'visit_id','Variation', 'clnt_tenure_yr',
       'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal',
       'calls_6_mnth', 'logons_6_mnth', 'date', 'process_step']], on=['client_id', 'visit_id'], how='left')

In [134]:
# Get last step per visit_id
final_df_c['last_step'] = final_df_c.groupby('visit_id')['process_step'].transform('last')

# Add new columns 1st_step, 2nd_step, 3rd_step for counts of step_1, step_2, step_3
step_counts = df_control.groupby(['visit_visitor_id', 'process_step']).size().unstack(fill_value=0)

# Add 1st_step, 2nd_step, 3rd_step columns to final_df
final_df_c['visit_visitor_id'] = final_df_c['visit_id'].astype(str) + "_" + final_df_c['visitor_id'].astype(str)
final_df_c['start_step'] = final_df_c['visit_visitor_id'].map(step_counts.get('start', pd.Series())).fillna(0).astype(int)
final_df_c['1st_step'] = final_df_c['visit_visitor_id'].map(step_counts.get('step_1', pd.Series())).fillna(0).astype(int)
final_df_c['2nd_step'] = final_df_c['visit_visitor_id'].map(step_counts.get('step_2', pd.Series())).fillna(0).astype(int)
final_df_c['3rd_step'] = final_df_c['visit_visitor_id'].map(step_counts.get('step_3', pd.Series())).fillna(0).astype(int)

# Create completion column
final_df_c['completion'] = final_df_c['last_step'].apply(lambda x: 1 if x == 'confirm' else 0)

# Create navigations_bt_start_last column
final_df_c['navigations_bt_start_last'] = final_df_c['1st_step'] + final_df_t['2nd_step'] + final_df_t['3rd_step']

In [135]:
# Drop visit_visitor_id column if no longer needed
final_df_c.drop(['visit_id', 'visitor_id', 'process_step', 'confirm_time'], axis=1, inplace=True)
final_df_c.drop_duplicates(subset='visit_visitor_id', keep='first', inplace=True)

In [136]:
# Reorder columns to place navigations_bt_start_last after 3rd_step and completed at the end
df_control_final = final_df_c.reindex(columns=[
    'client_id', 'visit_visitor_id', 'start_time', 'step_1', 'step_2', 'step_3', 
    'date', 'start_step', '1st_step', '2nd_step', '3rd_step', 'navigations_bt_start_last', 'last_step', 'completion',
    'total_time_visit', 'Variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 
    'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'
])

In [137]:
df_control_final = convert_data_types(df_control_final)

In [138]:
df_control_final.dtypes

client_id                            object
visit_visitor_id                     object
start_time                          float64
step_1                              float64
step_2                              float64
step_3                              float64
date                         datetime64[ns]
start_step                            int64
1st_step                              int64
2nd_step                              int64
3rd_step                              int64
navigations_bt_start_last             int64
last_step                            object
completion                             bool
total_time_visit                    float64
Variation                            object
clnt_tenure_yr                        int64
clnt_tenure_mnth                      int64
clnt_age                              int64
gendr                                object
num_accts                             int64
bal                                 float64
calls_6_mnth                    

### Create the csv file to df_test_final

In [139]:
df_control_final.to_csv('../Data/Cleaned_Data/df_control_final.csv', index=False)

# New Data Frames

## df_control_final

In [140]:
df_control_final.head(10)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,date,start_step,1st_step,2nd_step,3rd_step,navigations_bt_start_last,last_step,completion,total_time_visit,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,1028,557292053_87239438319_391157_42237450_62128060588,0.82,1.57,4.73,1.85,2017-04-08,1,5,2,1,7,step_1,False,8.97,Control,12,145,36,M,3,103520.22,1,4
9,1104,543158812_46395476577_767725_194240915_1815800...,0.00,0.00,0.00,0.00,2017-06-12,1,0,0,0,0,start,False,0.00,Control,5,66,48,Other,3,154643.94,6,9
10,1104,643221571_99977972121_69283_194240915_18158000533,0.00,0.00,0.00,0.00,2017-06-20,1,0,0,0,0,start,False,0.00,Control,5,66,48,Other,3,154643.94,6,9
11,1186,507052512_11309370126_442139_446844663_3161510...,0.00,0.00,0.00,0.00,2017-04-08,1,0,0,0,0,start,False,0.00,Control,8,99,22,Other,2,31662.52,0,3
12,1186,795373564_99931517312_810896_446844663_3161510...,0.18,0.18,0.00,0.00,2017-04-08,1,1,1,0,0,step_2,False,0.37,Control,8,99,22,Other,2,31662.52,0,3
15,1195,393817425_39015278493_996341_766842522_6999255...,0.55,1.63,0.52,1.38,2017-04-05,1,1,1,1,0,confirm,True,4.08,Control,21,262,54,M,2,28457.96,2,5
20,1197,71862471_21202285428_848395_753759429_54481946928,0.07,0.10,0.95,0.47,2017-04-01,1,1,2,2,0,confirm,True,1.58,Control,7,85,28,Other,5,177062.14,5,8
27,1368,784065271_45379483290_309335_366307863_1901466...,0.00,0.00,0.00,0.00,2017-04-09,1,0,0,0,0,start,False,0.00,Control,5,60,45,F,3,61956.25,6,9
28,2439,848231744_22569944243_37711_607208067_70160939111,0.00,0.00,0.00,0.00,2017-03-23,1,0,0,0,0,start,False,0.00,Control,11,142,56,M,2,504142.55,0,3
29,2581,182925466_27021409208_83502_770616558_80928163524,0.00,0.00,0.00,0.00,2017-04-09,1,0,0,0,0,start,False,0.00,Control,4,58,22,M,2,24976.79,1,4


## df_test_final

In [141]:
df_test_final.head(10)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,date,start_step,1st_step,2nd_step,3rd_step,navigations_bt_start_last,last_step,completion,total_time_visit,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,555,637149525_38041617439_716659_402506806_5608737...,0.12,0.53,1.65,0.33,2017-04-15,1,1,1,1,3,confirm,True,2.63,Test,3,46,29,Other,2,25454.66,2,6
5,647,40369564_40101682850_311847_66758770_53988066587,0.12,0.30,3.15,2.72,2017-04-12,1,1,1,1,3,confirm,True,6.28,Test,12,151,57,M,2,30525.80,0,4
10,934,7076463_57954418406_971348_810392784_45004760546,2.37,0.00,0.00,0.00,2017-04-18,4,0,0,0,0,start,False,2.37,Test,9,109,51,F,2,32522.88,0,3
14,1336,583743392_96265099036_939815_920624746_3260333...,0.42,0.43,0.85,1.82,2017-05-08,1,1,1,1,3,confirm,True,3.52,Test,48,576,42,M,4,130537.18,6,9
19,1336,614001770_19101025926_112779_920624746_3260333...,0.00,0.00,0.00,0.00,2017-05-08,0,0,0,0,0,confirm,True,1.37,Test,48,576,42,M,4,130537.18,6,9
21,1346,27144337_83739845380_214282_123474046_4204671056,2.65,3.37,0.82,1.42,2017-06-06,5,2,1,2,5,step_3,False,8.25,Test,14,177,46,F,2,822512.91,3,6
31,1516,255400977_38039535960_779641_182314299_6316858...,0.93,0.58,10.12,4.27,2017-04-06,1,1,1,1,3,confirm,True,15.90,Test,12,150,58,F,2,25408.39,5,8
36,1516,481123290_95510977345_707323_182314299_6316858...,1.33,1.67,1.08,0.00,2017-04-05,5,3,3,0,6,step_1,False,4.08,Test,12,150,58,F,2,25408.39,5,8
47,1643,753205700_16851596206_134483_121807985_2052456...,1.00,0.27,9.70,0.50,2017-06-10,1,1,1,1,3,confirm,True,11.47,Test,19,229,55,M,2,81731.39,0,3
52,1643,379946188_1773022140_107963_910842861_51062379179,0.00,0.00,0.00,0.00,2017-04-15,1,0,0,0,0,start,False,0.00,Test,19,229,55,M,2,81731.39,0,3


### Create the csv file to combine_df

In [142]:
df_final = pd.concat([df_test_final, df_control_final], ignore_index=True)

In [143]:
df_final = convert_data_types(df_final)

In [144]:
df_final.to_csv('../Data/Cleaned_Data/df_final.csv', index=False)

In [145]:
df_final.sample(5)

,client_id,visit_visitor_id,start_time,step_1,step_2,step_3,date,start_step,1st_step,2nd_step,3rd_step,navigations_bt_start_last,last_step,completion,total_time_visit,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
2998,792775,891936963_25562402727_806402_129419845_3153178...,0.32,1.73,0.58,1.65,2017-03-30,1,1,1,1,3,confirm,True,4.28,Test,10,129,64,M,3,490035.10,6,9
2085,536510,856497350_48415777304_156489_941370492_2213635...,0.00,0.00,0.00,0.00,2017-04-02,0,0,0,0,0,confirm,True,0.00,Test,5,70,51,Other,2,53137.18,1,4
15137,4080376,208010028_67616318750_39831_766102853_58415353419,0.53,1.07,1.28,2.65,2017-04-13,1,1,1,1,3,confirm,True,5.53,Test,27,331,71,F,3,671714.45,1,4
23111,6238365,318968175_85161387032_120276_691580687_4486079...,0.08,0.23,2.42,0.28,2017-06-18,1,1,1,1,3,confirm,True,3.02,Test,4,59,31,Other,3,41133.20,1,4
37537,120872,452784540_27907770905_546629_396753504_8696365...,0.42,0.00,0.00,0.00,2017-04-04,1,1,0,0,0,step_1,False,0.42,Control,15,187,64,M,3,187496.06,6,9


In [146]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69303 entries, 0 to 69302
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   client_id                  69303 non-null  object        
 1   visit_visitor_id           69303 non-null  object        
 2   start_time                 69303 non-null  float64       
 3   step_1                     69303 non-null  float64       
 4   step_2                     69303 non-null  float64       
 5   step_3                     69303 non-null  float64       
 6   date                       69303 non-null  datetime64[ns]
 7   start_step                 69303 non-null  int64         
 8   1st_step                   69303 non-null  int64         
 9   2nd_step                   69303 non-null  int64         
 10  3rd_step                   69303 non-null  int64         
 11  navigations_bt_start_last  69303 non-null  int64         
 12  last

In [147]:
df_control_final['last_step'].value_counts()

last_step
confirm    15336
start       9639
step_1      3498
step_3      2249
step_2      1459
Name: count, dtype: int64

In [152]:
df_final[['Variation','last_step']].value_counts()

Variation  last_step
Test       confirm      21435
Control    confirm      15336
           start         9639
Test       start         9448
Control    step_1        3498
Test       step_1        3136
Control    step_3        2249
Test       step_3        1782
Control    step_2        1459
Test       step_2        1321
Name: count, dtype: int64